In [3]:
from nytimesarticle import articleAPI
api = articleAPI('d8aa3577610f4275970e3db46f3f1f51')

In [25]:
articles = api.search( begin_date = 20170406, end_date=20170407 )

In [26]:
articles.keys()

dict_keys(['response', 'status', 'copyright'])

In [27]:
articles['response']['meta']

{'hits': 1803, 'offset': 0, 'time': 40}

In [29]:
type(articles['response']['docs'])

list

In [31]:
len(articles['response']['docs'])

10

In [34]:
type(articles['response']['docs'][0])

dict

In [35]:
articles['response']['docs'][0].keys()

dict_keys(['web_url', 'snippet', 'lead_paragraph', 'abstract', 'print_page', 'blog', 'source', 'multimedia', 'headline', 'keywords', 'pub_date', 'document_type', 'news_desk', 'section_name', 'subsection_name', 'byline', 'type_of_material', '_id', 'word_count', 'slideshow_credits'])

In [36]:
articles['response']['docs'][0]

{'_id': '58e7ed147c459f24986d9131',
 'abstract': None,
 'blog': [],
 'byline': {'organization': 'REUTERS', 'original': 'By REUTERS', 'person': []},
 'document_type': 'article',
 'headline': {'main': 'Latin America Nations Urge U.N.-Backed Solution to Syria Conflict',
  'print_headline': 'Latin America Nations Urge U.N.-Backed Solution to Syria Conflict'},
 'keywords': [],
 'lead_paragraph': 'Seven Latin American nations jointly expressed on Friday their concern about the escalation of violence in Syria and "strongly condemned the inhumane use of chemical weapons in that nation against civilians, particularly children."',
 'multimedia': [],
 'news_desk': 'None',
 'print_page': 0,
 'pub_date': '2017-04-07T19:48:28+0000',
 'section_name': 'World',
 'slideshow_credits': None,
 'snippet': 'Seven Latin American nations jointly expressed on Friday their concern about the escalation of violence in Syria and "strongly condemned the inhumane use of chemical weapons in that nation against civilia

In [5]:
print(articles)

{'response': {'meta': {'hits': 10834, 'time': 62, 'offset': 0}, 'docs': [{'web_url': 'https://www.nytimes.com/slideshow/2015/01/01/opinion/sunday/the-strip-2015.html', 'snippet': 'Cartoons about current affairs....', 'lead_paragraph': 'Cartoons about current affairs.', 'abstract': None, 'print_page': None, 'blog': [], 'source': 'The New York Times', 'multimedia': [{'width': 190, 'url': 'images/2012/07/08/opinion/sunday/the-strip-slide-KP3A/the-strip-slide-KP3A-thumbWide.png', 'height': 126, 'subtype': 'wide', 'legacy': {'wide': 'images/2012/07/08/opinion/sunday/the-strip-slide-KP3A/the-strip-slide-KP3A-thumbWide.png', 'wideheight': '126', 'widewidth': '190'}, 'type': 'image'}, {'width': 600, 'url': 'images/2012/07/08/opinion/sunday/the-strip-slide-KP3A/the-strip-slide-KP3A-articleLarge.png', 'height': 312, 'subtype': 'xlarge', 'legacy': {'xlargewidth': '600', 'xlarge': 'images/2012/07/08/opinion/sunday/the-strip-slide-KP3A/the-strip-slide-KP3A-articleLarge.png', 'xlargeheight': '312'},

In [7]:
def parse_articles(articles):
    '''
    This function takes in a response to the NYT api and parses
    the articles into a list of dictionaries
    '''
    news = []
    for i in articles['response']['docs']:
        dic = {}
        dic['id'] = i['_id']
        if i['abstract'] is not None:
            dic['abstract'] = i['abstract'].encode("utf8")
        dic['headline'] = i['headline']['main'].encode("utf8")
        dic['desk'] = i['news_desk']
        dic['date'] = i['pub_date'][0:10] # cutting time of day.
        dic['section'] = i['section_name']
        if i['snippet'] is not None:
            dic['snippet'] = i['snippet'].encode("utf8")
        dic['source'] = i['source']
        dic['type'] = i['type_of_material']
        dic['url'] = i['web_url']
        dic['word_count'] = i['word_count']
        # locations
        locations = []
        for x in range(0,len(i['keywords'])):
            if 'glocations' in i['keywords'][x]['name']:
                locations.append(i['keywords'][x]['value'])
        dic['locations'] = locations
        # subject
        subjects = []
        for x in range(0,len(i['keywords'])):
            if 'subject' in i['keywords'][x]['name']:
                subjects.append(i['keywords'][x]['value'])
        dic['subjects'] = subjects   
        news.append(dic)
    return(news)

In [8]:
parse_articles(articles)

[{'date': '2015-01-01',
  'desk': 'Opinion / Sunday Review',
  'headline': b'The Strip \xe2\x80\x94 2015',
  'id': '56af6dad38f0d82f06c0ce5f',
  'locations': [],
  'section': 'Opinion',
  'snippet': b'Cartoons about current affairs....',
  'source': 'The New York Times',
  'subjects': ['Comic Books and Strips',
   'United States Politics and Government'],
  'type': 'Slideshow',
  'url': 'https://www.nytimes.com/slideshow/2015/01/01/opinion/sunday/the-strip-2015.html',
  'word_count': '4'},
 {'date': '2015-01-01',
  'desk': 'Home & Garden',
  'headline': b'Installing Molding and Tin Ceilings',
  'id': '54dd119238f0d8448bfa34db',
  'locations': [],
  'section': 'Home & Garden',
  'snippet': b'Here\xe2\x80\x99s what you\xe2\x80\x99ll need....',
  'source': 'The New York Times',
  'subjects': ['Do It Yourself',
   'Interior Design and Furnishings',
   'Restoration and Renovation'],
  'type': 'Interactive Feature',
  'url': 'https://www.nytimes.com/interactive/2015/01/01/garden/12Pragmatist

In [32]:
def get_articles(sdate,edate):
    '''
    This function accepts a year in string format (e.g.'1980')
    and a query (e.g.'Amnesty International') and it will 
    return a list of parsed articles (in dictionaries)
    for that year.
    '''
    all_articles = []
    for i in range(0,100): #NYT limits pager to first 100 pages. But rarely will you find over 100 pages of results anyway.
        page_no=str(i)
        print(type(page_no))
        articles = api.search(
               fq = {'source':['The New York Times']},
               begin_date = sdate,
               end_date = edate,
               sort='oldest',
               page = page_no)
        articles = parse_articles(articles)
        all_articles = all_articles + articles
    return(all_articles)

